In [ ]:
# avoiding package version difference inconsistency for the dashboard
import werkzeug
from werkzeug.debug.tbtools import DebugTraceback
werkzeug.debug.tbtools.get_current_traceback = DebugTraceback

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
train_data_path = r"C:\Users\hasan\OneDrive\Masaüstü\Data_Science\Data_Science_Projects\Customer_Churn_Dataset\data\customer_churn_dataset-training-master.csv"
train_data = pd.read_csv(train_data_path)

In [ ]:
test_data_path = r"C:\Users\hasan\OneDrive\Masaüstü\Data_Science\Data_Science_Projects\Customer_Churn_Dataset\data\customer_churn_dataset-testing-master.csv"
test_data = pd.read_csv(test_data_path)

In [ ]:
data = train_data.append(test_data)

In [ ]:
data[data.loc[:,'CustomerID'].isna() == True]

In [ ]:
data.drop(index=199295, axis=1, inplace=True)

In [ ]:
data.reset_index(inplace=True)

In [ ]:
data.drop(columns='index',inplace=True)

In [ ]:
data.drop(columns='CustomerID', axis=1, inplace=True)

In [ ]:
for i in data.columns:
    if data.loc[:,i].dtypes == 'float64':
        data.loc[:,i] = data.loc[:,i].astype('int')

In [ ]:
# initialize app
app = dash.Dash(__name__)

In [ ]:
# dropdown menu container dictionaries in the list
dropdown_menu_list = []
j = 0

for i in data.columns:
    dropdown_menu_list.append({'label':i, 'value':i})
    j+=1

In [ ]:
# setting app layout
app.layout = html.Div(children=[
    
    # title division
    html.Div(html.H1('Customer Churn Dashboard', 
                     style={'textAlign':'center', 
                            'font-family': 'bahnschrift',
                            'color':'#839192'}),
             style={'padding':'1%'}
            ),
    
    
    
    # dropdown division
    html.Div([html.P('Please select an attribute in the dataset in order to analyze with respect to histogram plot ...', 
                     style={'textAlign':'middle',
                           'color':'#839192'}),
        
            dcc.Dropdown(id='attribute-dropdown',
                          options=dropdown_menu_list,
                          value='Churn',
                          placeholder = 'Please select an attribute in the dataset in order to analyze with respect to histogram plot ...')],
            
             style={'backgroundColor':'#F2F2F2', 
                   'border-radius': '9px',
                   'marginTop':'0.5%',
                   'marginBottom':'2%',
                   'marginLeft':'10%',
                   'marginRight':'10%'}),
    
    
    
    
    # plots' division
    html.Div(children = [
        
        # plot_1 division
        html.Div(dcc.Graph(id='plot-1'), 
                 style={'backgroundColor':'#FFFFFF', 
                        'display': 'inline-block',
                        'border-radius': '9px',
                        'marginRight':'1%',
                        'marginBottom':'2%',
                        'padding':'2%'}),
        
        # plot_2 division
        html.Div(dcc.Graph(id='plot-2'),
                 style={'backgroundColor':'#FFFFFF', 
                        'display': 'inline-block',
                        'border-radius': '9px',
                        'marginLeft':'1%',
                        'marginBottom':'2%',
                        'padding':'2%'})
        
    ], style={'backgroundColor':'#F2F2F2',
              'marginLeft':'5%',
              'marginRight':'5%',
              'marginTop':'2%',
              'marginBottom':'2%',
              'textAlign':'center',})
    
    
], style={'backgroundColor':'#F2F2F2', 
          'border-radius': '9px',
          'margin':'2%'})

In [ ]:
@app.callback(
    Output(component_id='plot-1', component_property='figure'),
    Output(component_id='plot-2', component_property='figure'),
    Input(component_id='attribute-dropdown', component_property='value')
)

def plottings(attribute):
    
    if attribute[0] == 'C' and attribute[-1] == 'n':
        
        data_churn = data['Churn'].value_counts().reset_index()
        fig = go.Figure(data=[go.Pie(labels=data_churn['index'], values=data_churn['Churn'], pull=[ 0.025, 0], hole=.2)])
        fig.update_layout(title_text=attribute+' Data Distribution Over The Dataset', width=500, height=500)
        
        fig_2 = px.histogram(data, x=attribute, color=attribute, width=500, height=500)
        fig_2.update_layout(title_text=attribute+' Data Distribution', bargap=0.2)
        
        return fig, fig_2
    
    else:
        churned_fig = px.histogram(data[data.loc[:,'Churn'] == 1], x=attribute, width=500, height=500)
        churned_fig.update_layout(title_text=attribute+' On Churned Dataset', bargap=0.1)
        
        unchurned_fig = px.histogram(data[data.loc[:,'Churn'] == 0],x=attribute, width=500, height=500)
        unchurned_fig.update_layout(title_text=attribute+' On UN-Churned Dataset', bargap=0.1)
        
        return churned_fig, unchurned_fig

In [ ]:
# run the dashboard
if __name__ == "__main__":
    app.run_server(port=1234)